# ランキング学習（LGBMRanker）による勝率分析
- 参考：https://qiita.com/kazuktym/items/3aa1f9e10d9231cf5cab#%E3%83%AC%E3%83%BC%E3%82%B9%E3%81%AF%E7%9B%B8%E5%AF%BE%E8%A9%95%E4%BE%A1%E3%81%A7%E4%BA%88%E6%83%B3%E3%81%97%E3%81%9F%E3%81%84

In [19]:
import pandas as pd
import tqdm

import os
import sys
sys.path.append('../../')
import common.com as common
from common import const,utils
import copy
from datetime import datetime
import scraping.scraping as scraping


In [20]:
# DBにある全てのレースデータを重複を排除して取得（2024/06/01実行時点では2013-01_2023-12のデータ）
_data = common.get_race_data_distinct()
data=pd.DataFrame(_data,columns=const.ORIGIN_RACE_DATA_HEADERS)

In [21]:
data=pd.read_csv('D://work//Programming//data//race_datas//2013-01_2023-12.csv',header=0)

In [22]:
# ラベルエンコーディング(LabelEncoder)
from sklearn.preprocessing import LabelEncoder

encode_tar_cols = [
'性',
'斤量',
'芝かダートか',
'回り',
'馬場状態',
'天気',]

encoded = copy.deepcopy(data)

le = LabelEncoder()
for col in encode_tar_cols:
    encoded[col] = le.fit_transform(encoded[col].values)

In [23]:
# 着順関連度で重み付け
ORDER_RELATED = {1:30,2:28,3:26,4:24,5:22}
def get_order_related(order):
    try:
        return ORDER_RELATED[order]
    except KeyError:
        return 0
    
encoded['着順関連度'] = encoded['着順'].map(get_order_related)

In [24]:
encoded['オッズ'] = encoded['オッズ'].replace('---',0).astype(float)

 30%|██▉       | 160714/536725 [00:00<00:00, 841801.81it/s]

---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---


 64%|██████▍   | 345599/536725 [00:00<00:00, 834990.97it/s]

---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---


100%|██████████| 536725/536725 [00:00<00:00, 809922.25it/s]

---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---


In [26]:

# objectになってしまうので、floatに変換
for col in const.VARIABLE[:-1]:
    if encoded[col].dtype == 'object':
        if col in ['レースID','馬番','枠番','齢','馬のID','騎手のID','距離','競馬場ID']:
            encoded[col] = encoded[col].astype('int64')
        else:
            encoded[col] = encoded[col].astype(float)

ValueError: could not convert string to float: '---'

In [ ]:
# レースIDでグループ化
grouped_data = encoded.groupby('レースID')

In [ ]:
# 8:2の割合で、学習用と評価用に分割
grouped_data_list = list(grouped_data)
learn_count = round(len(grouped_data_list) * 0.8)

learn_grouped_data = grouped_data_list[:learn_count]
test_grouped_data = grouped_data_list[learn_count:]

# 8:2の割合で、訓練用と検証用に分割
train_count = round(len(learn_grouped_data) * 0.8)
train_grouped_data = learn_grouped_data[:train_count]
valid_grouped_data = learn_grouped_data[train_count:]

In [ ]:
X_train = pd.concat([item[1].loc[:,const.VARIABLE] for item in train_grouped_data]).drop('着順関連度', axis=1)
X_valid = pd.concat([item[1].loc[:,const.VARIABLE] for item in valid_grouped_data]).drop('着順関連度', axis=1)
X_test = pd.concat([item[1].loc[:,const.VARIABLE] for item in test_grouped_data]).drop('着順関連度', axis=1)
y_train = pd.concat([item[1].loc[:,'着順関連度'] for item in train_grouped_data])
y_valid = pd.concat([item[1].loc[:,'着順関連度'] for item in valid_grouped_data])
y_test = pd.concat([item[1].loc[:,'着順関連度'] for item in test_grouped_data])

In [ ]:
# クエリーデータ(レース単位のデータ数のリスト)を作成
train_query = X_train.groupby('レースID').size().values.tolist()
valid_query = X_valid.groupby('レースID').size().values.tolist()

In [ ]:
import lightgbm as lgb

model = lgb.LGBMRanker(
  random_state=100,              # 乱数シード
  n_estimators=500,              # 決定木の個数(default:100)
  learning_rate=0.01,            # 学習率(default:0.1)
  num_leaves=100,                 # 決定木にある分岐の個数(default:31)
  max_depth=-1,                  # 決定木の深さの最大値(default:-1)
  min_child_samples=150,         # 一つの葉に含まれる最小データ数(default:20) 
)

model.fit(X_train, y_train,
  group=train_query,             # 訓練用クエリーデータ
  eval_set=[(X_valid, y_valid)], # 学習時に用いる検証用データ
  eval_group=[valid_query],      # 検証用クエリーデータ
  eval_metric='ndcg',            # 学習時の評価手法
  eval_at=[1, 2, 3,4,5]              # 学習時の評価対象順位
)


ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: オッズ: object

In [ ]:
y_pred = model.predict(X_test, num_iteration=model.best_iteration_)

# 予測結果や関連度をDataFrameに連結
df_pred = pd.DataFrame({
    'レースID': X_test['レースID'],
    '馬番号': X_test['馬のID'],
    '予想スコア':  y_pred,
    '着順関連度': y_test,
})

In [ ]:
# 特徴量重要度の抽出
df_importances = pd.DataFrame({'columns':X_train.columns, 'importances':model.feature_importances_})
df_importances.sort_values('importances', ascending=False, inplace=True)
print(df_importances)

In [ ]:
from sklearn.metrics import ndcg_score
# クエリーごとにNDCGを計算し、その平均値を算出
ndcg_score = df_pred.groupby('レースID').apply(lambda d: ndcg_score([d['着順関連度']], [d['予想スコア']], k=5)).mean()
print(ndcg_score)

In [ ]:
import pickle
# モデルの保存
file_dir = const.BASE_DIR+'LGBMRanker01.pickle'
pickle.dump(model, open(file_dir, 'wb'))